<H1>Web Crawler<H1>

# imports


In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# Main 

In order to find the urls in the main page we should:<br>
* Check with request that we can read the webpage.
* Check for  errors.
* Check that it has more then 10 following links.
* Colect the data.
* Export to excel.

## Test use case

* In order to find the links we should find the "< a >" in the page using BeautifulSoup library. <br>
* Then i'll check that it has more then 10 href in it and that they have a name.
* find the text data and expot it.


In [2]:
test_site = 'https://www.ynet.co.il/home/0,7340,L-8,00.html'
#test_site = 'https://crawler-test.com'

In [3]:

# Fetch the webpage
response = requests.get(test_site)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <a> tags and get their href attributes
links = soup.find_all('a', href=True)  # Filter out tags without href attributes

In [4]:
urls = []
for link in links:
    href = link.get('href')
    urls.append(href)


In [5]:
urls[0]

'https://www.ynet.co.il/news/category/184'

In [6]:
url_list = []
name_lst = []
text_list = []

To save

In [7]:
for i in urls[:20]:
    try:
        # Fetch the webpage
        response = requests.get(i)
        response.raise_for_status()
        if href:
        # Join the base URL with the relative URL
            full_url = urljoin(i, href)
            urls.append(full_url)

        # Parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        #print(soup)
        soup.find_all()
        # Safely get the title
        if soup.title and soup.title.string:
            name = soup.title.string
        else:
            name = 'No title found'

        #print(name)
        # Extract the text (or continue with other processing)
        page_text = soup.get_text()  
        #print(page_text)     
     
        url_list.append(i)
        name_lst.append(name)
        text_list.append(page_text)

    except requests.RequestException as e:
        continue

In [8]:
df = pd.DataFrame({
    'URL': url_list,
    'name':name_lst,
    'Text': text_list
})
#print(df)
# Save the DataFrame to an Excel file
df.to_excel('urls_and_texts_test.xlsx', index=False)


## Genral Case

Note that I limit the max href to prevent excesive runtime


In [9]:

def get_urls_in_page(main_url):
    try:
        # Fetch the main page
        print(f"Processing URL: {main_url}")

        response = requests.get(main_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all valid links
        links = soup.find_all('a', href=True)
        urls = [urljoin(main_url, link['href']) for link in links if link['href'].startswith(('http', '/'))]
        
        if len(urls) < 10:
            print(f"Less than 10 links found on {main_url}. Skipping.")
            return
        
        url_list = []
        name_list = []
        text_list = []
        
        for url in urls[:50]:  # Limiting to 50 URLs for processing
            try:
                
                # Fetch the webpage
                response = requests.get(url)
                response.raise_for_status()
                
                # Parse the HTML
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Safely get the title
                if soup.title and soup.title.string:
                    name = soup.title.string
                else:
                    name = 'No title found'
                
                # Extract the text content of the page
                page_text = soup.get_text()
                
                # Store the data
                url_list.append(url)
                name_list.append(name)
                text_list.append(page_text)
                
            except requests.RequestException as e:
                print(f"Error processing {url}: {e}")
                continue
        
        # Create DataFrame
        df = pd.DataFrame({
            'URL': url_list,
            'Name': name_list,
            'Text': text_list
        })
        
        # Save the DataFrame to an Excel file
        df.to_excel('href_name_text_general.xlsx', index=False)
        print("Data has been saved to href_name_text_general.xlsx")
        return df
        
    except requests.RequestException as e:
        print(f"Error fetching the main page {main_url}: {e}")




In [ ]:
get_urls_in_page('https://www.ynet.co.il/home/0,7340,L-8,00.html')

In [11]:
def on_button_click(b):  # Accepts a parameter 'b', which is passed by the button widget
    url = text_widget.value
    if url:

        # Process the URLs
        get_urls_in_page(url)
        
# Widget for URL input
text_widget = widgets.Textarea(
    value='',
    placeholder='Paste your URLs here, one per line',
    description='URLs:',
    disabled=False,
    layout=widgets.Layout(width='500px', height='100px')
)

# Button to trigger processing
button = widgets.Button(description="Process URL")
button.on_click(on_button_click)

# Display the widgets
display(text_widget, button)

Textarea(value='', description='URLs:', layout=Layout(height='100px', width='500px'), placeholder='Paste your …

Button(description='Process URL', style=ButtonStyle())

Processing URL: https://www.ynet.co.il/home/0,7340,L-8,00.html
Data has been saved to href_name_text_general.xlsx
